In [1]:
import polars as pl

import utils

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
num_tests: list[int] = [200, 500]
accuracies_home_dir = "accuracies_from_paper"

In [3]:
dfs: list[pl.DataFrame] = []
for num_test in num_tests:
    df = utils.load_all_accuracies(accuracies_home_dir, num_test).with_columns(
        num_test=pl.lit(num_test)
    )
    df = df.select("num_test").with_columns(df.select(pl.exclude("num_test")))
    dfs.append(df)

In [4]:
accuracy_df = pl.concat(dfs).sort(["num_test", "lm_type", "dataset"])

In [5]:
utils._summarize_differences(
    accuracy_df.with_columns(diff=pl.col("extra") - pl.col("base")),
    groups=("num_test", "lm_type"),
)

num_test,lm_type,mean,se
i32,str,f64,f64
200,"""bert""",0.040692,0.003457
200,"""gpt2""",0.062892,0.003096
500,"""bert""",0.061296,0.003077
500,"""gpt2""",0.038868,0.002498


In [6]:
utils._summarize_differences(
    accuracy_df.with_columns(diff=pl.col("test") - pl.col("extra")),
    groups=("num_test", "lm_type"),
)

num_test,lm_type,mean,se
i32,str,f64,f64
200,"""bert""",0.003264,0.004472
200,"""gpt2""",-0.000112,0.001463
500,"""bert""",-0.001592,0.004206
500,"""gpt2""",-0.002076,0.001527
